In [25]:
import sys
sys.path.append("/mnt/shared/Huggingface/sharedcode/Stegastamp_CR")
sys.path.append("/data/shared/Huggingface/sharedcode/Stegastamp_CR")
from utils.helpers import *
from dataset import CustomImageFolder
from torchvision import transforms
from functools import partial
import torch
from functions import *
from torchvision.datasets import ImageFolder
from tqdm.auto import tqdm
import csv

set_seeds(2024)
mi = "mi"
device = torch.device("cuda:0")
for model_choice in ["hidden"]:
    for data_choice in ["CelebA"]:
        data_dir = {"COCO":"/data/shared/coco2017/test2017","CelebA":"/data/shared/deepfake/CelebA-HQ/val"}
        ckp_dir = f"/data/shared/Huggingface/sharedcode/Stegastamp_Train/{mi}weights/{data_choice}/{model_choice}"
        transform_pipe = [
            transforms.Resize((128,128)),
            transforms.ToTensor(),
        ]
        normalize_ = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
        un_normalize_ = transforms.Normalize([-1,-1,-1],[2,2,2])
        if model_choice == 'hidden':
            transform_pipe.append(transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5]))
        transform = transforms.Compose(transform_pipe)
        if data_choice == "CelebA":
            ds = ImageFolder(data_dir[data_choice],transform=transform)
        else:
            ds = CustomImageFolder(data_dir[data_choice],transform,num=1000)

        # functions
        get_wmimages = partial(get_wmimages,ds=ds,device=device,model_choice=model_choice)
        get_images = partial(get_images,ds=ds,device=device)
        extract_message = partial(extract_message,device=device,model_choice=model_choice)
        csv_file = f"/data/shared/Huggingface/sharedcode/Stegastamp_CR/xyattackresults/oriresults/{data_choice}_{model_choice}_{mi}.csv"

        # attack
        message_len = 100
        # ckp_list = sorted(os.listdir(ckp_dir))
        ckp_list = [f"{mi}OO"]
        ckp_list = [f"{mi}emperical"]
        ckp_list = [f"{mi}GN_0.1"]
        # ckp_list = [f"{mi}affine_0.01"]

        
        if f"{mi}OO" in ckp_list:
            ckp_list.remove(f"{mi}OO")
            ckp_list.insert(0,f"{mi}OO")
        # ckp_list = ["OO","GN_0.1",'GN_0.25','GN_0.5',"affine_0.01","affine_0.02","affine_0.03"]
        for op in ckp_list:
            if op == "OO":
                ckp_path = os.path.join(ckp_dir,op,'epoch_499_state100.pth')
            elif op == 'emperical':
                ckp_path = os.path.join(ckp_dir,op,'epoch_499_state.pth')
            else:
                ckp_path = os.path.join(ckp_dir,op,'epoch_99_state.pth')

            encoder,decoder = load_weights(ckp_path,model_choice,message_len)
            encoder.eval(),decoder.eval()
            targe_message = generate_message(message_len,batch_size=1)
            for image_i in range(1000):
                wm_images,residual_predictions = get_wmimages(text=targe_message,image_i=range(image_i,image_i+1),encoder=encoder)
                residual_predictions_m = residual_predictions.mean(dim=0)
                # similate 
                init_message = "0" * message_len
                temp = init_message
                for bit in tqdm(range(message_len)):
                    temp1 = temp[:bit] + "1" + temp[bit+1:]
                    temp_tensor = generate_message(message_len,temp,batch_size=1)
                    temp1_tensor = generate_message(message_len,temp1,batch_size=1)
                    _,residual_predictions_0 = get_wmimages(text=temp_tensor,image_i=range(image_i,image_i+1),encoder=encoder)
                    _,residual_predictions_1 = get_wmimages(text=temp1_tensor,image_i=range(image_i,image_i+1),encoder=encoder)
                    
                    residual_predictions_0_m = residual_predictions_0.mean(dim=0)
                    residual_predictions_1_m = residual_predictions_1.mean(dim=0)

                    assert residual_predictions_0_m.size() == residual_predictions_1_m.size() == (3,128,128)

                    dis0 = torch.norm(residual_predictions_m - residual_predictions_0_m).item()
                    dis1 = torch.norm(residual_predictions_m - residual_predictions_1_m).item()
                    if dis0 < dis1:
                        temp = temp
                    else:
                        temp = temp1
                
                bit_acc = (message_len -  hamming_distance(temp,msg2str(targe_message))) / message_len
                csv_dict = {"model":op,"bit_acc":bit_acc}
                # with open(csv_file, mode='a', newline='') as file:
                #     fieldnames = csv_dict.keys()
                #     writer = csv.DictWriter(file, fieldnames=fieldnames)
                #     if file.tell() == 0:
                #         writer.writeheader()
                #     writer.writerow(csv_dict)
                print("op:",op)
                print("temp:",temp)
                print("target:",msg2str(targe_message))
                print("hamming distance with temp with target_message:",hamming_distance(temp,msg2str(targe_message)))  



100%|██████████| 100/100 [00:02<00:00, 35.42it/s]


op: miGN_0.1
temp: 1111110010011111110011000010100011000011001100001111111101100100101110011001000011100000010010000000
target: 0101011100001111011010100001101011010011001110001110111001100011101100011011010111011000011110000011
hamming distance with temp with target_message: 32


100%|██████████| 100/100 [00:02<00:00, 48.48it/s]


op: miGN_0.1
temp: 1111110010011101110011100010110011000000001000101010111001000000001100011001000011001000011010000001
target: 0101011100001111011010100001101011010011001110001110111001100011101100011011010111011000011110000011
hamming distance with temp with target_message: 32


100%|██████████| 100/100 [00:02<00:00, 47.67it/s]


...

op: miGN_0.1
temp: 1111000110011111110010000000100011000011001000001111111101100001101101111000001001001000011000000000
target: 0101011100001111011010100001101011010011001110001110111001100011101100011011010111011000011110000011
hamming distance with temp with target_message: 30


100%|██████████| 100/100 [00:02<00:00, 37.46it/s]


op: miGN_0.1
temp: 1111000110001101010010000000100011100011101000100111011001100100100101011001000001011000010000000000
target: 0101011100001111011010100001101011010011001110001110111001100011101100011011010111011000011110000011
hamming distance with temp with target_message: 33


  8%|▊         | 8/100 [00:00<00:02, 32.00it/s]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
def process_csv(file1, file2, output_file,score):
    # 读取两个CSV文件
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    if score == 'forgedacc' or score == 'xyscore':
        score = 'bit_acc'
    # 去掉第二个文件中的`mi`前缀
    df2['model'] = df2['model'].str.replace('mi', '', regex=False)
    
    # 提取 `bit_acc` 列并重命名
    bit_acc1 = df1[['model', score]].rename(columns={score: 'wo_bit_acc'})
    bit_acc2 = df2[['model', score]].rename(columns={score: 'w_bit_acc'})
    
    # 合并数据
    merged_df = pd.merge(bit_acc1, bit_acc2, on='model', how='left')
    
    # 如果在第二个文件中没有出现的模型，其 w_bit_acc 设为 wo_bit_acc
    merged_df['w_bit_acc'].fillna(merged_df['wo_bit_acc'], inplace=True)
    
    # 格式化结果
    merged_df['wo_bit_acc'] = merged_df['wo_bit_acc'].apply(lambda x: f"{x:.4f}")
    merged_df['w_bit_acc'] = merged_df['w_bit_acc'].apply(lambda x: f"{x:.4f}")
    
    # 保存到新的CSV文件
    merged_df.to_csv(output_file, index=False)
    

score = "xyscore"
for dataset in ["COCO","CelebA"]:
    for model in ["stega"]:
        csv_file1 = f"/data/shared/Huggingface/sharedcode/Stegastamp_CR/xyattackresults/{dataset}_{model}_.csv"
        csv_file2 = f"/data/shared/Huggingface/sharedcode/Stegastamp_CR/xyattackresults/{dataset}_{model}_mi.csv"
        output_file = f'/data/shared/Huggingface/sharedcode/Stegastamp_CR/xyattackresults/comparsion_{dataset}_{model}_{score}.csv'
        process_csv(csv_file1, csv_file2, output_file,score)